In [16]:
import os
import requests
import json
from openai import OpenAI
from dotenv import load_dotenv
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display, HTML


In [ ]:
 # take environment variables from .env.
load_dotenv()
print(os.getenv("OPENAI_API_KEY"))
api_key = os.getenv("OPENAI_API_KEY")


if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [18]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [19]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

In [20]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [21]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [22]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [12]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [23]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [24]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/tencent/Hunyuan-A13B-Instruct',
 '/google/magenta-realtime',
 '/google/gemma-3n-E4B-it',
 '/nanonets/Nanonets-OCR-s',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/OmniGen2/OmniGen2',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/tencent/Hunyuan3D-2.1',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/institutional/institutional-books-1.0',
 '/datasets/facebook/seamless-interaction',
 '/datasets/EssentialAI/essential-web-v1.0',
 '/datasets/FreedomIntelligence/ShareGPT-4o-Image',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammar

In [25]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}

Making Boucher for Company

In [26]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [27]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
black-forest-lab

In [28]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [29]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [33]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nblack-forest-labs/FLUX.1-Kontext-dev\nUpdated\n1 day ago\n•\n12.9k\n•\n896\ntencent/Hunyuan-A13B-Instruct\nUpdated\n1 day ago\n•\n489\ngoogle/magenta-realtime\nUpdated\n6 days ago\n•\n401\ngoogle/gemma-3n-E4B-it\nUpdated\n2 days ago\n•\n5.55k\n•\n242\nnanonets/Nanonets-OCR-s\nUpdated\n9 days ago\n•\n202k\n•\n1.24k\nBrowse 1M+ models\nSpaces\nRunning\n8.79k\n8.79k\nDeepSite v2\n🐳\nGenerate 

In [35]:
def create_brochure1(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [36]:
create_brochure1("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}


```markdown
# Hugging Face Brochure

## Welcome to Hugging Face

**The AI community building the future.**  
At Hugging Face, we are creating a collaborative platform that allows members of the machine learning (ML) community to come together to develop and share models, datasets, and applications. With over 1 million models and a plethora of datasets, we aim to accelerate progress in artificial intelligence.

---

## Our Offerings

### Models
Browse through an extensive catalog of cutting-edge models, including:
- **FLUX.1-Kontext-dev**
- **Hunyuan-A13B-Instruct**
- **magenta-realtime**
- **Nanonets-OCR-s**

With hundreds of thousands of models at your fingertips, you can harness the power of machine learning.

### Datasets
Access a diverse library of datasets tailored for various ML tasks, featuring:
- **awesome-chatgpt-prompts**
- **institutional-books-1.0**
- **facebook/seamless-interaction**

We provide tools to facilitate creating, discovering, and collaborating on ML tools.

---

## Company Culture

At Hugging Face, our culture is rooted in collaboration, innovation, and community engagement. We encourage our community members to share their knowledge and contribute to the evolving landscape of artificial intelligence. Whether you're a newbie or an expert, we foster an inclusive environment where every idea is valued. Our commitment to open-source development ensures that we are building the future of ML together.

---

## Join Our Community

### Customers
More than **50,000 organizations** harness the power of Hugging Face, including industry giants such as:
- Google
- Microsoft
- Amazon
- Grammarly

These organizations utilize our models and tools to drive their AI initiatives forward.

### Careers at Hugging Face
We are always on the lookout for talented individuals who are passionate about AI and ML. If you are eager to be part of a dynamic team that is shaping the future of technology, we invite you to explore our latest job openings. Benefits of joining our team include:
- Working with cutting-edge technologies
- Collaborating with industry pioneers
- Open and transparent workplace culture

---

## Get Involved

Whether you want to **sign up** to access our services, **explore** AI applications, or join our **community forums**, Hugging Face is here to support you at every turn.

**Explore our services:**    
- [Models](https://huggingface.co/models)    
- [Datasets](https://huggingface.co/datasets)    
- [Spaces](https://huggingface.co/spaces)    

## Get In Touch

For more information:
- Visit our website: [huggingface.co](https://huggingface.co)
- Follow us on [Twitter](#), [LinkedIn](#), or join our [Discord](#).

---

**Hugging Face**  
*Together, let’s build the future of AI.*
```

Samll improvment

In [37]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [38]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face!

**The AI Community Building the Future**

At Hugging Face, we are dedicated to forging the future of artificial intelligence through community collaboration. Our innovative platform serves as a hub where the machine learning community can come together to share models, datasets, and applications, providing an inclusive environment for both professionals and enthusiasts to explore the vast potential of AI.

---

## Our Offerings:

- **Models**: Over **1 million models** available at your fingertips! Explore trending models such as FLUX.1, Hunyuan-A13B, and many more.
- **Datasets**: Access to **250,000+ datasets**, enabling you to find the right data for your machine learning projects effortlessly.
- **Spaces**: Create and collaborate on applications housed within our platform, powered by **400,000+ applications** running on state-of-the-art technology.

---

## Who We Serve

More than **50,000 organizations** trust Hugging Face, including industry giants like Google, Amazon, Microsoft, and Meta. Our clientele ranges from research institutions to large enterprises, demonstrating the adaptability and reach of our tools across various sectors in AI and ML.

---

## Company Culture

At Hugging Face, our culture is rooted in collaboration, innovation, and inclusivity. We believe in the power of community-driven progress and actively encourage the participation of diverse voices in AI. We foster a supportive environment that nurtures talent, creativity, and continuous learning, making it a prime place for individuals passionate about AI to thrive.

---

## Careers at Hugging Face

Join us in our mission to build the future of AI! We are always on the lookout for talent that values collaboration and innovation. Positions are open across various domains, from engineering and research to community management and enterprise support. As a member of our team, you will work at the forefront of machine learning technology and have opportunities to develop and grow in your career.

---

## Join Us Today!

Ready to be a part of this vibrant AI community? 

- [Explore our models](https://huggingface.co/models)
- [Join our community](https://huggingface.co/community)
- [Apply for a job](https://huggingface.co/jobs)

Together, let’s build the future of AI with purpose and passion!

--- 

For more information, visit our website: [Hugging Face](https://huggingface.co) 

In [41]:
def format_brochure_content1(content):
    """
    Format the brochure content with beautiful styling
    """
    # Convert markdown to HTML with custom styling
    formatted = content
    
    # Add custom styling for different elements
    formatted = formatted.replace("## ", '<h2 style="color: #667eea; border-bottom: 2px solid #667eea; padding-bottom: 10px; margin-top: 30px;">')
    formatted = formatted.replace("### ", '<h3 style="color: #764ba2; margin-top: 25px;">')
    formatted = formatted.replace("**", '<strong style="color: #667eea;">')
    formatted = formatted.replace("**", '</strong>')
    
    # Style bullet points
    formatted = formatted.replace("- ", '<li style="margin: 8px 0; padding-left: 10px;">• ')
    formatted = formatted.replace("\n- ", '</li><li style="margin: 8px 0; padding-left: 10px;">• ')
    
    # Add icons for common sections
    formatted = formatted.replace("About", "🏢 About")
    formatted = formatted.replace("Services", "🛠️ Services")
    formatted = formatted.replace("Products", "📦 Products")
    formatted = formatted.replace("Contact", "📞 Contact")
    formatted = formatted.replace("Mission", "🎯 Mission")
    formatted = formatted.replace("Vision", "��️ Vision")
    
    # Style paragraphs
    paragraphs = formatted.split('\n\n')
    styled_paragraphs = []
    for p in paragraphs:
        if p.strip() and not p.startswith('<'):
            styled_paragraphs.append(f'<p style="margin: 15px 0; text-align: justify;">{p}</p>')
        else:
            styled_paragraphs.append(p)
    
    formatted = '\n\n'.join(styled_paragraphs)
    
    return formatted

In [40]:
def stream_brochure1(company_name, url):
    """
    Stream a beautiful brochure with enhanced formatting and styling
    """
    # Create a beautiful header
    header_html = f"""
    <div style="
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 20px;
        border-radius: 15px;
        margin-bottom: 20px;
        box-shadow: 0 8px 32px rgba(0,0,0,0.1);
        text-align: center;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    ">
        <h1 style="margin: 0; font-size: 2.5em; font-weight: 300; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
            🏢 {company_name}
        </h1>
        <p style="margin: 10px 0 0 0; font-size: 1.1em; opacity: 0.9;">
            📄 AI-Generated Company Brochure
        </p>
        <div style="margin-top: 15px; padding: 10px; background: rgba(255,255,255,0.1); border-radius: 8px;">
            <small>🔗 Source: {url}</small>
        </div>
    </div>
    """
    
    # Display the header
    display(HTML(header_html))
    
    # Create a progress indicator
    progress_html = """
    <div style="
        background: #f8f9fa;
        border: 2px solid #e9ecef;
        border-radius: 10px;
        padding: 15px;
        margin: 20px 0;
        text-align: center;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    ">
        <div style="display: flex; align-items: center; justify-content: center; gap: 10px;">
            <div class="spinner" style="
                width: 20px;
                height: 20px;
                border: 3px solid #f3f3f3;
                border-top: 3px solid #667eea;
                border-radius: 50%;
                animation: spin 1s linear infinite;
            "></div>
            <span style="color: #6c757d; font-weight: 500;">🤖 AI is crafting your brochure...</span>
        </div>
    </div>
    
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    </style>
    """
    
    display(HTML(progress_html))
    
    # Start the stream
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )
    
    response = ""
    display_handle = display(HTML(""), display_id=True)
    
    # Process the stream with enhanced formatting
    for chunk in stream:
        if chunk.choices[0].delta.content:
            response += chunk.choices[0].delta.content
            
            # Clean and format the response
            formatted_response = format_brochure_content(response)
            
            # Create beautiful content wrapper
            content_html = f"""
            <div style="
                background: white;
                border-radius: 15px;
                padding: 30px;
                margin: 20px 0;
                box-shadow: 0 4px 20px rgba(0,0,0,0.08);
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                line-height: 1.6;
                color: #333;
            ">
                {formatted_response}
            </div>
            """
            
            # Update the display
            if display_handle:
                display_handle.update(HTML(content_html))
    
    # Add a beautiful footer
    footer_html = """
    <div style="
        background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
        color: white;
        padding: 20px;
        border-radius: 15px;
        margin-top: 20px;
        text-align: center;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        box-shadow: 0 8px 32px rgba(0,0,0,0.1);
    ">
        <div style="display: flex; align-items: center; justify-content: center; gap: 15px;">
            <span style="font-size: 1.2em;">✨</span>
            <span style="font-weight: 500;">Brochure generated successfully!</span>
            <span style="font-size: 1.2em;">✨</span>
        </div>
        <p style="margin: 10px 0 0 0; opacity: 0.9; font-size: 0.9em;">
            Powered by AI • Generated with ❤️
        </p>
    </div>
    """
    
    display(HTML(footer_html))
    
    return response

def format_brochure_content(content):
    """
    Format the brochure content with beautiful styling
    """
    # Convert markdown to HTML with custom styling
    formatted = content
    
    # Add custom styling for different elements
    formatted = formatted.replace("## ", '<h2 style="color: #667eea; border-bottom: 2px solid #667eea; padding-bottom: 10px; margin-top: 30px;">')
    formatted = formatted.replace("### ", '<h3 style="color: #764ba2; margin-top: 25px;">')
    formatted = formatted.replace("**", '<strong style="color: #667eea;">')
    formatted = formatted.replace("**", '</strong>')
    
    # Style bullet points
    formatted = formatted.replace("- ", '<li style="margin: 8px 0; padding-left: 10px;">• ')
    formatted = formatted.replace("\n- ", '</li><li style="margin: 8px 0; padding-left: 10px;">• ')
    
    # Add icons for common sections
    formatted = formatted.replace("About", "🏢 About")
    formatted = formatted.replace("Services", "🛠️ Services")
    formatted = formatted.replace("Products", "📦 Products")
    formatted = formatted.replace("Contact", "📞 Contact")
    formatted = formatted.replace("Mission", "🎯 Mission")
    formatted = formatted.replace("Vision", "��️ Vision")
    
    # Style paragraphs
    paragraphs = formatted.split('\n\n')
    styled_paragraphs = []
    for p in paragraphs:
        if p.strip() and not p.startswith('<'):
            styled_paragraphs.append(f'<p style="margin: 15px 0; text-align: justify;">{p}</p>')
        else:
            styled_paragraphs.append(p)
    
    formatted = '\n\n'.join(styled_paragraphs)
    
    return formatted

In [42]:
stream_brochure1("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


"# Hugging Face: The AI Community Building the Future\n\n---\n\n## Overview\nHugging Face is at the forefront of the AI and machine learning community, providing a platform that empowers collaboration on models, datasets, and applications. With over 1 million models and 250,000 datasets, Hugging Face is designed to facilitate the creation and exploration of cutting-edge AI technologies. The organization is dedicated to building a strong community around AI and machine learning, ensuring that everyone can contribute and benefit from shared knowledge.\n\n## Company Culture\nAt Hugging Face, we believe in the importance of community and collaboration. Our culture is characterized by:\n- **Open Source:** We are committed to building the foundation of machine learning tools together with our community. This includes our popular repositories like Transformers, Diffusers, and Tokenizers.\n- **Inclusivity:** We welcome contributions from diverse perspectives, ensuring that our platform reflect